In [2]:
import math, random, copy, os, glob, time
from itertools import chain, combinations, permutations
from pprint import pprint

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision as tv
from torchvision import datasets, transforms as T

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

from utils.parameters import args_parser
from utils.data_process import prepareIID, prepareNIID1, prepareNIID2, prepareNIID12
from models.Nets import SmallMLP_MNIST, MediumMLP_MNIST, LargeMLP_MNIST, SmallMLP_EMNIST, MediumMLP_EMNIST, LargeMLP_EMNIST
from utils.save_file import createDirectory, deleteAllModels, saveCheckpoint, print_parameters, loadCheckpoint
from models.Fed import FedAvg
from utils.helpers import powerset, grangerset, aggListOfDicts, getAllClients

from models.server import server
from models.clients import initClients

#新加的
from PIL import ImageFile
import torchvision

In [7]:

# Get cpu or gpu device for training 选择cpu或者gpu
device = "cuda" if torch.cuda.is_available() else "cpu"  
# Create subdirectories 创建存放模型的子目录
model_path = './models'
# Initialize system and define helper functions 初始化系统并定义助手功能
createDirectory(model_path)
# Delete existing .pt files from previous run 从上次运行中删除现有.pt文件
deleteAllModels(model_path)

In [8]:
#训练集与测试集
ImageFile.LOAD_TRUNCATED_IMAGES = True # To prevent error during loading broken images
PATH_TRAIN = "train"
PATH_TEST  = "test"

BATCH_SIZE = 32

TOTAL_SIZE = len(os.listdir(PATH_TRAIN + "/Normal")) + len(
os.listdir(PATH_TRAIN + "/Infected"))
TOTAL_TEST_SIZE = len(os.listdir(PATH_TEST + "/Normal")) + len(
os.listdir(PATH_TEST + "/Infected"))
STEPS_PER_EPOCH = TOTAL_SIZE // BATCH_SIZE
STEPS_PER_TEST_EPOCH = TOTAL_TEST_SIZE // BATCH_SIZE

IMAGE_H, IMAGE_W = 224, 224

In [9]:
transform = torchvision.transforms.Compose(
[  # Applying Augmentation
    torchvision.transforms.Resize((IMAGE_H, IMAGE_W)),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.RandomRotation(30),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
])  # Normalizing data
    
train_data = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)
test_data = torchvision.datasets.ImageFolder(root=PATH_TEST, transform=transform)

In [19]:
NUM_OF_CLIENTS = 5
train_datasets = prepareIID(train_data, NUM_OF_CLIENTS)

In [20]:
train_dataloaders = [DataLoader(train_dataset, batch_size=BATCH_SIZE) for train_dataset in train_datasets]
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE * 2) #batch_size批次大小

In [22]:
# Define network model architecture 定义网络模型体系结构
FederatedModel = None #联邦学习网络模型
FederatedModel = torchvision.models.resnet18(False)
feature = FederatedModel.fc.in_features
FederatedModel.fc = nn.Linear(feature, 2)
FederatedModel.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [31]:
FederatedLossFunc = nn.CrossEntropyLoss() #损失函数
FederatedOptimizer = torch.optim.AdamW(FederatedModel.parameters(), lr=1e-4)
FederatedLearnRate = 0.0001 #学习率
FederatedMomentum = 0.9 #动量
FederatedWeightDecay = 1e-5 #权重衰退

In [25]:
server = server(FederatedModel, FederatedLossFunc, FederatedOptimizer, FederatedLearnRate, FederatedMomentum,FederatedWeightDecay)

In [35]:
server = server.initServer(model_path, 'FedAvg', test_dataloader)

Initializing server model...


TypeError: 'AdamW' object is not callable

In [36]:
clients = initClients(4, 0, 1, server,train_dataloaders)

Initializing clients...


TypeError: 'server' object is not subscriptable